# FUNZIONI DASK_CUDF CHE NON VANNO (PER FARE CONFRONTO)

## Questa è la sezione da portare per far vedere i limiti di dask_cudf

### Conversione colonna da stringa a datetime

Errore che restituisce:
"TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy()."

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
a = base.split('gas_start_date', 2)
a = a.rename(columns=cols)
a['month'] = a['month'].str.lower().map(mesi)
dd.to_datetime(a).head()

### Conversione colonna da stringa in formato "1,22 unita_misura" a float

In [ ]:
#rimozione unità di misura
base.strip(['average_gas_bill_cost'], '€/smc')['average_gas_bill_cost']
#sostituzione della , con .
base.replace(['average_gas_bill_cost'], ',', '.', regex=True)['average_gas_bill_cost']

#### Le operazioni preliminari vanno a buon fine, qua si aprono più scenari (fallimentari)

questa soluzione è quella corretta su dask normale, utilizzando dask_cudf però genera un errore dovuto alla conversione implicita di un array in un numpy array, cosa non fattibile all'interno della GPU quindi consiglia di costruire esplicitamente una matrice tramite Cupy (Cuda Python) ma queste sono operazioni interne alla libreria

In [ ]:
dd.to_numeric(base.get_df()['average_gas_bill_cost'])
 

Qua da 2 diverse tipologie di errori: una dovuta ai metadati e l'altra alla presenza di valori non convertibili, anche togliedo tutti i valori nan con [~base['average_gas_bill_cost'].isna()] la situazione non cambia

In [ ]:
base.get_df()['average_gas_bill_cost'].map_partitions(cudf.to_numeric, 'raise')

# Pipeline

In [ ]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

In [ ]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

In [ ]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

# Loading Dataset

In [ ]:
import dask_cudf as dc
import dask.dataframe as dd
import dask.array as da
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cudf
import numpy as np
import graphviz
from base_dask_cudf import BaseDfBench
base = BaseDfBench()

In [ ]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

base.load_dataset('parquet')
df = base.get_df()
columns = base.get_columns()
base.npartitions()

In [ ]:
#verica di user_code+customer_code = PK utility
#PK customer = user_code
#PK invoice = bill_id
df.groupby(by=['user_code', 'customer_code'])['bill_id'].count().compute()

## Load big Dataset

In [ ]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

In [ ]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(f"DF partitions: {ddf.npartitions}")
print(f"DF: {ddf}")

# Funzioni utili

## Da portare

In [11]:
def col_type(df, find=['numeric']):
    """
    Return a list containing all columns of the specified type
    df : dataframe
    type : list or single value with the specified value/s
    """
    tipo = []
    if type(find) == list:
        tipo.extend(find)
    else:
        tipo.append(find)
    
    col = set() #no duplicate column name
    
    for i in tipo:
        if i == 'numeric':
            col.update(df.select_dtypes(include=np.number).columns)
        elif i == 'float':
            col.update(df.select_dtypes(include=float).columns)
        elif i == 'int':
            col.update(df.select_dtypes(include=int).columns)
        elif i == 'date':
            col.update(df.select_dtypes(include=np.datetime64).columns)
        elif i == 'string':
            col.update(df.select_dtypes(include=object).columns)
            
    return list(col)

In [12]:
col_type(base.get_df())

['bill_id',
 'gas_consumption',
 'light_transport_cost',
 'gas_system_charges',
 'light_consumption',
 'gas_offer',
 'light_offer',
 'F2_kWh',
 'light_system_charges',
 'light_average_cost',
 'light_material_cost',
 'gas_material_cost',
 'age',
 'F3_kWh',
 'F1_kWh',
 'gas_transport_cost']

In [ ]:
def get_duplicate_col_V2(df, soglia=0.9, nan=False):
    """
    General versione of get_duplicate_columns
    nan : if True the divisor doesn't consider nan value
    """
    cols = df
    duplicate_col = []
        
    for i in range(cols.shape[1]):
        for j in range(i+1, cols.shape[1]):
            divisore = cols.shape[0]
            if nan == True:
                if (cols.iloc[:,i].count() > cols.iloc[:,j].count()):
                    divisore = cols.iloc[:,i].count()
                else:
                    divisore = cols.iloc[:,j].count()
            if (cols.iloc[:,i].dtype == cols.iloc[:,j].dtype) and ((cols.iloc[:,i] == cols.iloc[:,j]).sum() / float(divisore) >= soglia):
                duplicate_col.append((cols.iloc[:,i].name, cols.iloc[:,j].name))
                
    return duplicate_col

In [ ]:
def get_duplicate_columnss(df):
    cols = df
    duplicate_col = []

    for i in range(cols.shape[1]):
        for j in range(i+1, cols.shape[1]):
            if cols.iloc[:,i].equals(cols.iloc[:,j]):
                duplicate_col.append((cols.iloc[:,i].name, cols.iloc[:,j].name))

    return duplicate_col

In [ ]:
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}

def convert_to_datetime(col): 
    if col.dtype == object:
        a = cudf.DataFrame()
        a[['day', 'month', 'year']] = col.str.split(expand=True)
        a['month'] = a.month.str.lower().map(mesi)
        col = cudf.to_datetime(a)
    return col

In [ ]:
#CONVERSIONE COLONNE SU DASK_CUDF 
mesi = {
    'gennaio': '01',
    'febbraio': '02',
    'marzo' : '03',
    'aprile': '04',
    'maggio': '05',
    'giugno': '06',
    'luglio': '07',
    'agosto': '08',
    'settembre': '09',
    'ottobre': '10',
    'novembre': '11',
    'dicembre': '12',
}
cols = {
    0:'day',
    1:'month',
    2:'year'
}
a = base.split('gas_start_date', 2)
a = a.rename(columns=cols)
a['month'] = a['month'].str.lower().map(mesi)
dd.to_datetime(a).head()

## Da non portare, DIREI

In [ ]:
# Dovrebbe accorgersi di colonne con similarità > soglia
# TODO --> FUNZIONE DA RIFARE PERCHé USA I SET E QUINDI NON CONFRONTA RIGA-RIGA 
#          MA SET(ELEMENTI_COL1) CON SET(eLEMENTI_COL2)
def get_duplicate_col(df, soglia=0.9):
        """
        Return a list of duplicate columns, if exists.
        Duplicate columns are those which have same values for each row.
        """  
        cols = df.columns.values
        #return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if self.df[cols[i]].equals(self.df[cols[j]])]
        
        return [(cols[i], cols[j]) for i in range(0, len(cols)) for j in range(i+1, len(cols)) if len(set(cols[i]) & set(cols[j])) / float(len(set(cols[i]) | set(cols[j]))) > soglia ]

# 1. INGESTION & DISCOVERY

## Analysis DA PORTARE

In [ ]:
#tipi di dato delle colonne
base.get_columns_types()

In [ ]:
'''
import re
#df['light_amount'].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)') 
print(cudf.to_numeric(df['light_amount'].str.replace(',', '.'), errors='coerce').isna().sum()  )
df['light_amount'].isna().sum()
'''

In [ ]:
#Ottenere unità di misura
base.split('average_gas_bill_cost', 1)[1].value_counts().index.compute()[0]

In [ ]:
# Rimozione unità di misura
base.strip(['average_gas_bill_cost'], '€/smc')['average_gas_bill_cost'].head()

In [ ]:
base.replace(['average_gas_bill_cost'], ',', '.', regex=True)['average_gas_bill_cost'].head()

In [ ]:
dd.to_numeric(base.get_df()['average_gas_bill_cost'])

In [ ]:
base.get_df()['average_gas_bill_cost'].map_partitions(cudf.to_numeric, 'raise').tail()

In [ ]:
base.get_df()['average_gas_bill_cost'] = dd.to_numeric(base.get_df()['average_gas_bill_cost'], errors='coerce')
base.get_df().head()

In [13]:
# Trovare le tipologie di colonne a seconda della tipologia
int_cols = col_type(df, find=['int'])
string_cols = col_type(df, find=['string']) #object
float_cols = col_type(df, find=['float'])
numeric_cols = col_type(df, find=['numeric'])
date_cols = col_type(df, find=['date'])

In [15]:
#esempio
base.locate_null_values('address').head()

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
1611,1611,23311.0,2219.0,467.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,8CE314D06FEFFD54C0F1497C20C479F376102747,...,<NA>,F631ED0EF8EC71428366F15C369AA53792A6F20D,monthly,False,<NA>,1176.31,<NA>,481.81,<NA>,2067.89
1627,1627,210.0,149.0,226.0,VIETRI DI POTENZA,<NA>,<NA>,P,45,A1E7C992D46A2D6D911C3595DF603D3F2C6BEB37,...,<NA>,1E9BC1B19300B6E472980894714A4C8382C565EA,monthly,False,<NA>,76.08,<NA>,50.45,<NA>,85.57
1639,1639,95.0,52.0,105.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,3CE203296BE19C1257020DA7EFEB84A0D3F4F7B7,...,<NA>,CC5C0DB0AA0034008B3398775B1EC89FC21A9DDB,monthly,False,<NA>,40.80,<NA>,29.47,<NA>,45.65
1858,1858,65.0,34.0,60.0,VIETRI DI POTENZA,<NA>,<NA>,P,66,D3117D65827932EB6A7F54F73EB1700C0D9F289C,...,<NA>,EB24C1C62642D39E65FB5BA50044153D2BF6EEC9,monthly,False,<NA>,44.17,<NA>,36.34,<NA>,47.11
2511,2511,285.0,406.0,232.0,VIETRI DI POTENZA,<NA>,<NA>,P,0,44F5D3195791957FCF7010485404E83F3BDB9F0B,...,<NA>,1D2EAAAFDD212B85E05CC2275D44020B05B1E5B7,monthly,False,<NA>,101.28,<NA>,74.09,<NA>,87.20


## Locate Missing Values DA PORTARE

In [61]:
# LOCATE MISSING VALUES
#rows = df.shape[0]
#wait(ddf.shape[0])
#print("Total rows", rows)

# These are the colums with at least one missing value
column_count_nan = {}
# compute the presence of null value on time instead every iteration for performance reasons
na = base.get_df().isna()
val = na.any().compute()

for c, val in vec.to_pandas().items(): 
    count = na[c].sum()
    if (val):
        #print(c, "\t Affected rows:", count)
        column_count_nan[c] = count
    else:
        #print(c, "\t Qui no missing values")
        column_count_nan[c] = count

In [ ]:
column_with_nan = []

perc = 50
soglia = ((perc * base.get_df().shape[0]) / 100).compute()
print(soglia)

for c, val in column_count_nan.items():
    if val.compute() > soglia:
        #print(c['column_name'])
        column_with_nan.append(c)
        
column_with_nan

# Si può notare come nessuna colonna abbia il 100% di valori nulli, ma ci sono altresì tante 
# colonne con un numero elevato di nan. Poche colonne hanno un numero basso di nan.

### Analisi extra (utile estenderla all'intero dataset)

In [ ]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative')

In [ ]:
base.locate_null_values('address')

In [ ]:
df[df['city'] == 'VIETRI DI POTENZA']

In [ ]:
# PROVO A VEDERE SE SONO PRESENTI ALTRI RECORDS DEGLI UTENTI A CUI MANCANO NOMINATIVE E ADDRESS
v = df['user_code'].value_counts()
v.index[v.gt(1)]

In [ ]:
# DATASET CON GLI user_code CHE COMPAIONO PIù DI UNA VOLTA
df[df['user_code'].isin(v.index[v.gt(1)])]

In [ ]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

df.loc[0, 'billing_frequency']

## Locate Outliers DA PORTARE (qui c'è già roba con dask, da verificare se funziona)

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
from dcudf import DCUDF

#ddf = DCUDF("/data/invoices.csv", 0.1, True)
ddf = DCUDF()

In [ ]:
ddf.cluster

In [ ]:
ddf.client

In [ ]:
ddf.df

In [ ]:
ddf.get_columns_types()

In [ ]:
tmp = ddf.locate_outliers('F1_kWh')
tmp.compute()

In [ ]:
type = ddf.df.dtypes
num = type[type!='object'][type!='bool']
num = num.index.to_list()
num

In [ ]:
tmp = ddf.locate_outliers("F1_kWh")
tmp
# a lot of task per partition

In [ ]:
tmp = tmp.persist()
tmp
# no more than 1 task per partition

In [ ]:
tmp.shape

In [ ]:
tmp = ddf.locate_outliers("F1_kWh")
tmp = tmp.compute()
tmp.shape

In [ ]:
from tqdm import tqdm

for col in tqdm(num):
    tmp = ddf.locate_outliers(col)
    tmp = tmp.compute()
    
    print(f"{col} has {tmp.shape[0]} outliers")

## Sort Data DA PORTARE (qui c'è già roba con dask, da verificare se funziona)

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

from dcudf import DCUDF

#ddf = DCUDF("/data/invoices.csv", 0.1, True)
ddf = DCUDF()

In [ ]:
ddf.sort("F1_kWh").compute()
# not working

In [ ]:
tmp = ddf.sort_index("F1_kWh").compute()
# not working

In [ ]:
idx = tmp.index.to_arrow().to_pylist()
# too mutch long list

In [ ]:
tmp.merge(ddf.df, on=["F1_kWh"], how="left")

# 2. VALIDATION

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

## Check Data Range DA PORTARE TUTTE (mi tirava fare un sottogruppo)

### Age nulla per PIVA

In [ ]:
# Controllo se chi ha P ha età nulla
df.loc[(df['sex']=='P') & (df['age']>=0), 'age'] = np.nan
df.loc[(df['sex']=='P') & (np.isnan(df['age']))]

### Age >= 18

In [ ]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
#
df['age'].isna().sum()

In [ ]:
# Che fare con i minorenni? Li mettiamo tutti a 18 anni?
df[df['age'] < 18]

### Translation in English

In [ ]:
# Dobbiamo metterli in inglese? Si può fare con un map
ing = {
    'luce': 'light',
    'gas e luce': 'gas and light',
    'gas' : 'gas',
}
df['supply_type'] = df['supply_type'].map(ing)

In [ ]:
df['supply_type'].value_counts(dropna=False)

### billing_freq & bill_type Analysis

In [ ]:
# Che fare qua? Billing frequency (‘monthly’, ‘quarterly’, etc.)
# Si potrebbero calcolare
df['billing_frequency'].value_counts(dropna=False)

In [ ]:
# Kind of invoice (False means a “standard bill”)
df['bill_type'].value_counts(dropna=False)

### light_offer_type Analysis

In [ ]:
# Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
df['light_offer_type'].value_counts(dropna=False)

In [ ]:
# I 2 valori di light mi puzzano, famo uno strip?
df['light_offer_type'] = df['light_offer_type'].str.strip()
df['light_offer_type'].value_counts(dropna=False)
# GHAVEVO RAGIONE Vè

### Applico la funzione di strip a tutte le colonne con stringhe

In [ ]:
# STRIPPATINA GENERALE (alle colonne con stringhe)
df = base.strip(string_cols, ' ')

## Check Column Uniqueness DA PORTARE E DA VERIFICARE

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [ ]:
#provo a concatenare lo stesso dataset più volte solo per vedere come regge lo scaling di dati
big_df = cudf.concat([df for _ in range(100)], sort=False) #200k con la paglia

In [ ]:
base.get_duplicate_columnss()
# Ne vengono individuate solo 5 di coppie 100% fedeli, 
# ma cambia la questione se abbassiamo la soglia? Ci sono degli errori sopra perchè 
# quelle 100% uguali non vengono riconosciute, mentre qua sì

In [ ]:
# Versione con soglia. (Lentina!)
duplicate_columns = get_duplicate_col_V2(df, soglia=1, nan=True)
duplicate_columns

## Find Data Mismatch

In [ ]:
date_col_list = search_by_pattern('date')
for col in date_col_list:
    df[col] = convert_to_datetime(df[col])
df[date_col_list]

In [ ]:
df['date'] == df['light_start_date']

In [ ]:
# Seguendo il GroundTruth mystico
mappone = {'bill_id': 'int64',
 'F1_kWh': 'float64',
 'F2_kWh': 'float64',
 'F3_kWh': 'float64',
 'city': 'str',
 'address': 'str',
 'nominative': 'str',
 'sex': 'str',
 'age': 'int64',
 'user_code': 'str',
 'date': 'datetime64[s]',
 'light_start_date': 'datetime64[s]',
 'light_end_date': 'datetime64[s]',
 'tv': 'float64',
 'gas_amount': 'float64',
 'gas_average_cost': 'float64',
 'light_average_cost': 'float64',
 'emission_date': 'datetime64[s]',
 'supply_type': 'str',
 'gas_start_date': 'datetime64[s]',
 'gas_end_date': 'datetime64[s]',
 'extra_fees': 'float64',
 'gas_consumption': 'float64',
 'light_consumption': 'float64',
 'gas_offer': 'float64',
 'light_offer_type': 'str',
 'light_offer': 'str',
 'howmuch_pay': 'float64',
 'total_amount': 'float64',
 'light_amount': 'float64',
 'average_unit_light_cost': 'float64',
 'average_light_bill_cost': 'float64',
 'average_unit_gas_cost': 'float64',
 'average_gas_bill_cost': 'float64',
 'customer_code': 'str',
 'billing_frequency': 'str',
 'bill_type': 'str',
 'gas_system_charges': 'float64',
 'light_system_charges': 'float64',
 'gas_material_cost': 'float64',
 'light_transport_cost': 'float64',
 'gas_transport_cost': 'float64',
 'light_material_cost': 'float64'
}

In [ ]:
#Check sui tipi di dato attuali
type_dict = base.get_columns_types()
type_dict

In [ ]:
#poichè probabilmente con i <NA> ho un sacco di problemi "lasciatemi castaareeee.. 
#con la chitarra in maaanoooo, lasciatemi castare.. sono un italiano"

df = base.fill_nan(np.nan)
df

In [ ]:
# setto le colonne con i tipi di dato in mappone
df = base.cast_columns_types(mappone)

In [ ]:
df['emission_date']

In [ ]:
# Abbiamo dei valori non castabili a float nelle colonne interessate

# Trovare le tipologie di colonne a seconda della tipologia
int_cols = col_type(df, find=['int'])
string_cols = col_type(df, find=['string']) #object
float_cols = col_type(df, find=['float'])
numeric_cols = col_type(df, find=['numeric'])
date_cols = col_type(df, find=['date'])

# Vediamo cos'ho nelle colonne str
#print(string_cols)
for c in string_cols:
    print("Col: ", c)
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(cosa)

In [ ]:
# C'è della merda, ottimizzo la cosa riducendo il campo d'analisi a solo le colonne float64 di mappone
mappone_float64_cols = []

for (k, v) in mappone.items():
    if v == 'float64':
        mappone_float64_cols.append(k)

final_float_cols = [x for x in mappone_float64_cols if x not in float_cols]
mappone_float64_cols
float_cols
final_float_cols = [x for x in final_float_cols if x not in ['gas_offer']]

In [ ]:
# Altro caso di colonna dal dubbio gusto
df['gas_offer']

In [ ]:
# Vediamo cos'ho nelle colonne str
#print(string_cols)
lista_rename = []
for c in final_float_cols:
    cosa = df[c].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    if cosa != '0.00' and cosa != 'nan':
        print("Col: ", c + ' ' + cosa)
        print(cosa)
        p = {
            c : c + ' ' + cosa
        }
        lista_rename.append(p)
        df[c] = df[c].str.replace(',', '.').str.extract('-?([0-9]*\.[0-9]+)').astype(float)
df

In [ ]:
# voglio trovare le colonne str che diventeranno float che generano problemi
# VIRGOLE DI MERDA
problematiche = [x for x in string_cols if x in final_float_cols]
for p in problematiche:
    print(p)
    df[p] = df[p].str.replace(',', '.').str.split(expand=True).iloc[:,-1].value_counts().index[0]
    print(float(df[p][0]))

In [ ]:
float(0.48)

In [ ]:
# Ho diverse colonne da cui eliminare le unità di misura...
df['gas_average_cost'].dtype

In [ ]:
# in 'bill_type' False == 'standard bill'
# Funziona solo se la funzione sopra non va in errore.

df = base.replace('bill_type', 'False', 'standard bill', False)
df

In [ ]:
#Verifica della bontà della sostituzione
df['bill_type'].value_counts()

In [ ]:
# Non c'è nessuna riga tutta nulla, Giulio è contento
df.dropna(how='all')

# 3. STRUCTURING

In [ ]:
''' 
+++++ DATA STRUCTURING +++++
    • Change column data types
    • Delete, split or merge columns
    • Pivot and unpivot
'''

## Change Column Data Types 

In [ ]:
# aggregabile con lo step precedente
# L'HO FATTO SOPRA, è DA ORDINARE E PERFEZIONARE

## Delete, Split or Merge Columns

In [ ]:
# rimozione colonne duplicate 

## Pivot and Unpivot

In [ ]:
# non saprei cosa farci

# 4. ENRICHMENT

In [ ]:
''' 
+++++ DATA ENRICHMENT +++++
    • Calculate columns using expressions
    • Set primary key column
    • Join or append DataFrames
    • Group by and aggregate records
    • Scale column values into a certain range
    • Encode categorical data (one-hot encoding and label encoding)
'''

## Calculate Columns Using Expressions

## Set PK column

# 5. FILTERING

In [ ]:
''' 
+++++ DATA FILTERING +++++
    • Sample rows
    • Select a subset of rows through a user-defined query
'''

## Sample Rows

In [ ]:
df.sample(20)

In [ ]:
# SELECT A SUBSET OF ROWS?!?

# 6. CLEANING

In [ ]:
''' 
+++++ DATA CLEANING +++++
    • Change data format and case
    • Deduplicate data
    • Missing value imputation
    • Find and replace values
'''

In [ ]:
# CHANGE DATA FORMAT AND CASE
# data format dovrebbe già essere fatto a questo punto
# case pure se vogliamo aver fatto tutto per bene

In [ ]:
# DEDUPLICATE DATA
# controllo se ho più bollette uguali dello stesso cliente

In [ ]:
ddf

In [ ]:
# MISSING VALUE IMPUTATION
# qua? Spero di averne pochissimi nel caso

In [ ]:
# FIND AND REPLACE VALUES
# ossia?

In [ ]:
x = np.nan
float(x)